# A more complex example for modeling drug-protein interactions with asymmetric RAF kinase dimerization and two RAF inhibitors

Here, we propose a more complex example of modeling RAF kinases and RAF inhibitors by reproducing a model of asymmetric RAF kinase dimerization and combination of two RAF inhibitors. This model was originally developed in *Kholodenko BN (2015) Drug Resistance Resulting from Kinase Dimerization Is Rationalized by Thermodynamic Factors Describing Allosteric Inhibitor Effects. Cell Rep 12: 1939–1949*. In the paper, the model is shown in *Figure 3* and the corresponding rate derivations are in *Reaction List S3*. While in the original work the cooperative thermodynamic reaction rates were derived manually, here we use energy PySB (via energy BNG) to recreate the same reaction network structure.

First, we define the model:

In [197]:
from pysb import Model, Monomer, Parameter, Expression,  Rule, Observable, Initial, Annotation, EnergyPattern, ANY
from pysb.bng import generate_equations
from pysb.export import export
from pysb.core import as_complex_pattern, ComplexPattern
from sympy import exp, log

Model();
model.name='complex_example_asym_RAF_two_RAFi';

#define RAF kinase with a binding site for RAF (r) and a drug (i) that can be into two configurations when dimerized (R1,R2)
Monomer('R', ['r', 'i', 'state'], {'state':['none', 'R1', 'R2']});   
#define first RAF inhibitor with binding site for RAF 
Monomer('I1',['r']);   
#define second RAF inhibitor with binding site for RAF 
Monomer('I2',['r']);   

#define the initial conditions for R and I1,I2
Parameter('R_0',0.01);  # uM
Parameter('I1_0',0.0);  # uM
Parameter('I2_0',0.0);  # uM
Initial(R(r=None, i=None, state='none'), R_0);
Initial(I1(r=None), I1_0);
Initial(I2(r=None), I2_0);

#define kinetic parameters and rules for independent binding
#RAF dimerization
#define dissociation constant (kD), forward rate (kf) and distributionr rate (phi) for RAF dimerization
Parameter('kr_1',10);  #/s
Parameter('kf_1',1.0);  #/s/uM
Parameter('phi_1',1.0); #unitless 
#convert kinetic parameters into energies for RAF dimerization
Expression('Gf_1', log(kr_1/kf_1)); #unitless 
Expression('Ea0_1',-phi_1*log(kr_1/kf_1)-log(kf_1)); #unitless 
# define energy in bond between R and R
EnergyPattern('ep_1',R(r=1)%R(r=1),Gf_1);
#define RAF dimerization reaction with asymmetry of configurations R1 and R2
Rule('R1', R(r=None, state='none')+R(r=None, state='none') | R(r=1, state='R1')%R(r=1, state='R2') , phi_1, Ea0_1, energy=True);

#I1 binding to RAF
#define dissociation constant (kD), forward rate (kf) and distributionr rate (phi) for drug binding to RAF
Parameter('kr_2a',0.1);  #/s
Parameter('kf_2a',1.0);  #/s/uM
Parameter('phi_2a',1.0); #unitless
#convert kinetic parameters into energies for drug binding to RAF
Expression('Gf_2a', log(kr_2a/kf_2a)); #unitless 
Expression('Ea0_2a',-phi_2a*log(kr_2a/kf_2a)-log(kf_2a)); #unitless 
# define energy in bond between R and I
EnergyPattern('ep_2a',R(i=1)%I1(r=1),Gf_2a);
#define drug binding to RAF reaction
Rule('R2a', R(i=None)+I1(r=None) | R(i=1)%I1(r=1) , phi_2a, Ea0_2a, energy=True);

#I2 binding to RAF
#define dissociation constant (kD), forward rate (kf) and distributionr rate (phi) for drug binding to RAF
Parameter('kr_2b',0.1);  #/s
Parameter('kf_2b',1.0);  #/s/uM
Parameter('phi_2b',1.0); #unitless
#convert kinetic parameters into energies for drug binding to RAF
Expression('Gf_2b', log(kr_2b/kf_2b)); #unitless 
Expression('Ea0_2b',-phi_2b*log(kr_2b/kf_2b)-log(kf_2b)); #unitless 
# define energy in bond between R and I
EnergyPattern('ep_3',R(i=1)%I2(r=1),Gf_2b);
#define drug binding to RAF reaction
Rule('R2b', R(i=None)+I2(r=None) | R(i=1)%I2(r=1) , phi_2b, Ea0_2b, energy=True);

Next, we define the thermodynamic parameters and impose energy patterns to model cooperativity in drug-protein interactions:

In [198]:
#define thermodynamic factors
Parameter('fa',1.0);  #unitless 
Parameter('fb',1.0);  #unitless 
Parameter('g1a',1.0);  #unitless
Parameter('g1b',1.0);  #unitless
Parameter('g2a',1.0);  #unitless
Parameter('g2b',1.0);  #unitless
Parameter('g3a',1.0);  #unitless
Parameter('g3b',1.0);  #unitless

#convert thermodynamic factors in energies 
Expression('Gf_fa',log(fa)); #unitless 
Expression('Gf_fb',log(fb)); #unitless 
Expression('Gf_g1a',log(g1a)); #unitless 
Expression('Gf_g1b',log(g1b)); #unitless 
Expression('Gf_g2a',log(g2a)); #unitless 
Expression('Gf_g2b',log(g2b)); #unitless 
Expression('Gf_g3a',log(g3a)); #unitless 
Expression('Gf_g3b',log(g3b)); #unitless 

#define energy patterns for fa
EnergyPattern('ep_fa_RRI',R(r=1,i=None,state='R2')%R(r=1,i=2,state='R1')%I1(r=2), Gf_fa);
#define energy patterns for fb
EnergyPattern('ep_fb_RRI',R(r=1,i=None,state='R2')%R(r=1,i=2,state='R1')%I2(r=2), Gf_fb);
#define energy patterns for g1a
#Expression('Gf_g1a_fa_RRI', Gf_g1a + Gf_fa);
#EnergyPattern('ep_g1a_RRI',R(r=1,i=None,state='R1')%R(r=1,i=2,state='R2')%I1(r=2), Gf_g1a_fa_RRI);
#define energy patterns for g1b
#Expression('Gf_g1b_fb_RRI', Gf_g1b + Gf_fb);
#EnergyPattern('ep_g1b_RRI',R(r=1,i=None,state='R1')%R(r=1,i=2,state='R2')%I2(r=2), Gf_g1b_fb_RRI);


The thermodynamic parameters have the following interpretations: 
* *fa*: 
* *fb*:
* *g1q*:
* *g1b*:
* *g2a*:
* *g2b*:
* *g3a*:
* *g3b*:

We generate the kinetic model:

In [199]:
# generate the model equations
generate_equations(model)

# print model infomration
print ('Model information')
print ('Species:',len(model.species))
print ('Parameters:',len(model.parameters)+len(model.initial_conditions))
print ('Expressions:',len(model.expressions))
print ('Observables:', len(model.observables))
ntotr=len(model.rules);
nenergy=len([r for r in model.rules if r.energy]);
print ('Total Rules:', ntotr)
print ('Energy Rules:', nenergy)
print('Non-energy Rules:', ntotr-nenergy)
print('Energy Patterns:', len(model.energypatterns))
print('Reactions:',len(model.reactions))

#save the generated model in PySB and BNG format
generated_model_code = export(model, 'pysb_flat')
with open(model.name+'.py', 'wt') as f:
    f.write(generated_model_code);

generated_model_code = export(model, 'bngl')
with open(model.name+'.bngl', 'wt') as f:
    f.write(generated_model_code);

Model information
Species: 14
Parameters: 23
Expressions: 14
Observables: 0
Total Rules: 3
Energy Rules: 3
Non-energy Rules: 0
Energy Patterns: 5
Reactions: 46


To show the automatically generated equation rates we need to define support functions:

In [200]:
from sympy import Symbol

def transform_rate(r):
    # Expand PySB Expressions into their full contents.
    r = r.replace(
        lambda a: isinstance(a, Expression),
        lambda e: e.expand_expr()
    )
    # Eliminate Species symbols by replacing them with 1.
    r = r.replace(
        lambda a: isinstance(a, Symbol) and a.name.startswith('__s'),
        lambda s: 1
    ) 
    r = r.simplify()
    return r

def format_sp(species):
    # Return LaTeX-formated sum of species variables for given species numbers.
    # Turns a list like [1,3] into "$s_1$ + $s_3$".
    return " + ".join(f"$s_{{{i}}}$" for i in species)

def to_latex(e):
    return latex(e, mul_symbol="dot")

We inspect the species and reaction rates generated by the energy-based model formulation: 

In [201]:
from sympy import latex, simplify
import pandas as pd
import re
# Prevent pandas from truncating long LaTeX expressions when rendering.
pd.options.display.max_colwidth=1000

#display the species in the model 
speciesdisp = pd.DataFrame(
    data=[[f"\[s_{{{idx}}}\]", str(species)] for idx, species in enumerate(model.species)],
    columns=['ID', 'Pattern']
)
#display species and their patterns with all headers and cell contents centered, and no index.
display(
    speciesdisp.style
    .set_properties(**{"text-align": "center"})
    .set_table_styles([
        dict(selector="th", props=[("text-align", "center")]),
        dict(selector=".MathJax_Display", props=[("text-align", "center !important")]),
    ])
    .hide_index()
    .set_caption('SPECIES')
)

#display the reactions and reaction rules in the model
reactions_unidirectional = pd.DataFrame(model.reactions)
for c in "rule", "reverse":
    if (reactions_unidirectional[c].map(len) > 1).any():
        raise Exception("Cannot handle models that produce the same reaction from different rules")
    reactions_unidirectional[c] = reactions_unidirectional[c].map(lambda x: x[0])
reactions_unidirectional["K"] = reactions_unidirectional["rate"].map(transform_rate)
group_direction = reactions_unidirectional.groupby("reverse")
forward = group_direction.get_group(False)[["reactants", "products", "rule", "K"]]
reverse = group_direction.get_group(True)[["reactants", "products", "K"]]
reverse[["reactants", "products"]] = reverse[["products", "reactants"]]
reactions = pd.merge(forward, reverse, on=["reactants", "products"], suffixes=["f", "r"])
reactions["Kd"] = (reactions["Kr"] / reactions["Kf"]).apply(simplify)
rdisp = pd.DataFrame({
    "ID": [f"\[R_{{{i}}}\]" for i in range(len(reactions))],
    "Base rule": reactions["rule"],
    "Reaction": [f"{format_sp(r.reactants)} \u21c4 {format_sp(r.products)}" for r in reactions.itertuples()],
    "Forward rate ($k^+$ )": "\[" + reactions["Kf"].map(to_latex).str.replace('kf_', 'k^+_', regex=False) + "\]",
    "Backward rate ($k^-$ )": "\[" + reactions["Kr"].map(to_latex).str.replace('kr_', 'k^-_', regex=False) + "\]",
    "Dissociation constant ($K_d$ )": "\[" + reactions["Kd"].map(to_latex).str.replace('kf_', 'k^+_', regex=False)
                                       .str.replace('kr_', 'k^-_', regex=False)+ "\]",
})
# Display reactions with reaction rates with all headers and cell contents centered, and no index.
display(
    rdisp.style
    .set_properties(**{"text-align": "center"})
    .set_table_styles([
        dict(selector="th", props=[("text-align", "center")]),
        dict(selector=".MathJax_Display", props=[("text-align", "center !important")]),
    ])
    .hide_index()
    .set_caption('REACTIONS')
)

ID,Pattern
\[s_{0}\],"R(r=None, i=None, state='none')"
\[s_{1}\],I1(r=None)
\[s_{2}\],I2(r=None)
\[s_{3}\],"R(r=1, i=None, state='R1') % R(r=1, i=None, state='R2')"
\[s_{4}\],"I1(r=1) % R(r=None, i=1, state='none')"
\[s_{5}\],"I2(r=1) % R(r=None, i=1, state='none')"
\[s_{6}\],"I1(r=1) % R(r=2, i=1, state='R2') % R(r=2, i=None, state='R1')"
\[s_{7}\],"I2(r=1) % R(r=2, i=1, state='R2') % R(r=2, i=None, state='R1')"
\[s_{8}\],"I1(r=1) % R(r=2, i=1, state='R1') % R(r=2, i=None, state='R2')"
\[s_{9}\],"I1(r=1) % I1(r=2) % R(r=3, i=1, state='R1') % R(r=3, i=2, state='R2')"


ID,Base rule,Reaction,Forward rate ($k^+$ ),Backward rate ($k^-$ ),Dissociation constant ($K_d$ )
\[R_{0}\],R1,$s_{0}$ + $s_{0}$ ⇄ $s_{3}$,\[k^+_{1}\],\[k^-_{1}\],\[\frac{k^-_{1}}{k^+_{1}}\]
\[R_{1}\],R2a,$s_{0}$ + $s_{1}$ ⇄ $s_{4}$,\[k^+_{2a}\],\[k^-_{2a}\],\[\frac{k^-_{2a}}{k^+_{2a}}\]
\[R_{2}\],R2b,$s_{0}$ + $s_{2}$ ⇄ $s_{5}$,\[k^+_{2b}\],\[k^-_{2b}\],\[\frac{k^-_{2b}}{k^+_{2b}}\]
\[R_{3}\],R1,$s_{0}$ + $s_{4}$ ⇄ $s_{6}$,\[k^+_{1}\],\[k^-_{1}\],\[\frac{k^-_{1}}{k^+_{1}}\]
\[R_{4}\],R1,$s_{0}$ + $s_{5}$ ⇄ $s_{7}$,\[k^+_{1}\],\[k^-_{1}\],\[\frac{k^-_{1}}{k^+_{1}}\]
\[R_{5}\],R1,$s_{0}$ + $s_{4}$ ⇄ $s_{8}$,\[fa^{- \phi_{1}} \cdot k^+_{1}\],\[fa^{1 - \phi_{1}} \cdot k^-_{1}\],\[\frac{fa \cdot k^-_{1}}{k^+_{1}}\]
\[R_{6}\],R1,$s_{4}$ + $s_{4}$ ⇄ $s_{9}$,\[k^+_{1}\],\[k^-_{1}\],\[\frac{k^-_{1}}{k^+_{1}}\]
\[R_{7}\],R1,$s_{4}$ + $s_{5}$ ⇄ $s_{10}$,\[k^+_{1}\],\[k^-_{1}\],\[\frac{k^-_{1}}{k^+_{1}}\]
\[R_{8}\],R1,$s_{0}$ + $s_{5}$ ⇄ $s_{11}$,\[fb^{- \phi_{1}} \cdot k^+_{1}\],\[fb^{1 - \phi_{1}} \cdot k^-_{1}\],\[\frac{fb \cdot k^-_{1}}{k^+_{1}}\]
\[R_{9}\],R1,$s_{4}$ + $s_{5}$ ⇄ $s_{12}$,\[k^+_{1}\],\[k^-_{1}\],\[\frac{k^-_{1}}{k^+_{1}}\]
